In [3]:
#GENETIC ALGORITHM
import math
import random
import numpy as np
board_size = 5 # size of board and numebr of queens
num_states = 8 # number of states for genetic algorithm. MUST BE EVEN
max_attacking_pairs = 0 # maximum number of attacking pairs for a given board size. to be set using combination function

def combination(n,r):
    return math.factorial(n)/ (math.factorial(n-r)*math.factorial(r))
max_attacking_pairs = combination(board_size, 2)

def make_board(size):
    board = [[0 for j in range(size)] for i in range(size)]
    return board

def random_queen_placement(board):
    board = [[0 for j in range(len(board))] for i in range(len(board))]
    for i in range(len(board)):
        j = random.randint(0, len(board) - 1)
        board[i][j] = 1
    return board

"""RETURNS ARRAY OF POSITIONS OF QUEENS ON THE BOARD"""
def get_queens(board):
    queens = []
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == 1:
                queens.append([i, j])
    return queens

"""RETURNS AN ARRAY OF ALL SQUARES ATTACKED BY A SINGLE QUEEN"""
def get_attacking_squares(board, queen):
    #print("GET ATTACKING SQUARES:")
    attacks = []
    #print(queen)
    #print("VERTICAL") # y axis, first dimension
    for i in range(len(board)):
        if i != queen[0]:
            attacks.append([i, queen[1]])
    #print("HORIZONTAL") # x axis, second dimension
    for i in range(len(board)):
        if i != queen[1]:
            attacks.append([queen[0], i])
    #print("DIAGONAL")
    for i in range(1, len(board)): #down right
        if i + queen[0] >= len(board) or i + queen[1] >= len(board):
            break
        attacks.append([i + queen[0], queen[1] + i])
    for i in range(1, len(board)): #up right
        if queen[0] - i < 0 or queen[1] + i >= len(board):
            break
        attacks.append([queen[0] - i, queen[1] + i])
    for i in range(1, len(board)): #down left
        if i + queen[0] >= len(board) or queen[1] - i < 0:
            break
        attacks.append([i + queen[0], queen[1] - i])
    for i in range(1, len(board)): #up left
        if queen[0] - i < 0 or queen[1] - i < 0:
            break
        attacks.append([queen[0] - i, queen[1] - i])
    return attacks

"""DETERMINE HOW MANY PIECES(QUEENS) A QUEEN IS ATTACKING"""
def count_attacks(board, queen):
    count = 0
    a = get_attacking_squares(board, queen)
    for square in a:
        if board[square[0]][square[1]] == 1:
            #print("found queen at", square)
            count += 1
    return count

def fitness(board):
    queens = get_queens(board)
    a_count = 0
    for queen in queens:
        a_count += count_attacks(board, queen)
    return a_count//2

#GNERATE 8 STATES
states = []
def generate_initial_states(num_states, board_size):
    states = []
    for i in range(num_states):
        states.append(make_board(board_size))
        states[i] = random_queen_placement(states[i])
        # ADD FITNESS
        states[i] = [states[i], (int(max_attacking_pairs - fitness(states[i])))]
        # ADD PROBABILITIES
    states = generate_probabilities(states)
    for i in range(num_states):
        string = board_to_string(states[i][0])
        states[i] = [states[i][0], states[i][1], states[i][2], states[i][3], string]
    return states

def generate_probabilities(states):
    total_non_attacking_pairs = 0
    for i in range(len(states)):
        total_non_attacking_pairs += states[i][1]
    for i in range(len(states)):
        if total_non_attacking_pairs == 0: # even ditribution
            probability = 1 / num_states
            #print("DIVIDE BY ZERO AVERTED. UNIFORM DISTRIBUTION PROBABILITY:", probability)
            states[i] = [states[i][0], states[i][1], probability]
        else:
            probability = states[i][1] / total_non_attacking_pairs
            states[i] = [states[i][0], states[i][1], probability]
        #print(states[i])
    # RUNNING SUM OF PROBABILITIES
    running_sum = 0
    for i in range(len(states)):
        running_sum += states[i][2]
        states[i] = [states[i][0], states[i][1], states[i][2], running_sum]
    return states

def random_state_selection(states):
    selected_states = []
    for i in range(len(states)):
        r = random.uniform(0, 1) # random number
        selected_board = None
        for i in range(len(states)):
            if r < states[i][3]:
                selected_board = states[i]
                break
        #print("Selection", i, selected_board[1:], "r:", r)
        selected_states.append(selected_board)
    return selected_states

def board_to_string(board):
    string = ""
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 1:
                string += str(j)
    return string

def string_to_board(string):
    board = make_board(len(string))
    for i in range(len(board)):
        board[i][int(string[i])] = 1
    return board

def crossover(states):
    recombinants = []
    for i in range(0, len(states), 2):
        chiasm = cut_here(len(states[i+1][4]))
        string_A = states[i][4]
        string_B = states[i+1][4]
        #print(i, ":", string_A,  i+1, ":", string_B, "CHIASM:", chiasm) #print("THIS IS WHERE THE MAGIC HAPPENS: RECOMBINANT STATES") #print(string_A[:chiasm] + "|" + string_B[chiasm:]) #print(string_B[:chiasm] + "|" + string_A[chiasm:])
        recombinant_1 = string_A[:chiasm] + string_B[chiasm:]
        recombinant_2 = string_B[:chiasm] + string_A[chiasm:]
        recombinants.append(recombinant_1)
        recombinants.append(recombinant_2)
    return recombinants
        
"""SELECT A CHIASM (LOCATION IN THE MIDDLE OF STRING FOR CROSSOVER)"""
def cut_here(str_length):
    # divide the string in 3 approximately even pieces. cut in the middle piece. works for strings of length 4 or greater
    nums = [x for x in range(str_length)]
    one_third = math.ceil(len(nums) / 3)
    cut_nums = nums[one_third - 1 : 2 * one_third - 1]
    return random.choice(cut_nums)

def generate_successor_states(next_generation, board_size):
    states = []
    for i in range(len(next_generation)):
        states.append(next_generation[i])
        # ADD FITNESS
        states[i] = [states[i], (int(max_attacking_pairs - fitness(states[i])))]
        # ADD PROBABILITIES
        states = generate_probabilities(states)
    # GET STRING ENCODINGS
    for i in range(num_states):
        string = board_to_string(states[i][0])
        states[i] = [states[i][0], states[i][1], states[i][2], states[i][3], string]
    return states

"""RANDOM MUTATION WITH 50% PROBABILITY. NEW VALUE IS RANDOM, SO IT MIGHT BE SAME VALUE (A SILENT MUTATION)."""
def random_mutation(recombinants):
    for i in range(len(recombinants)):
        if random.randint(0, 1) == 1:
            r = random.randint(0, len(recombinants[i]) - 1)
            m = random.randint(0, len(recombinants[i]) - 1)
            #print("MUTATION", "at index:", r, "value:", m)
            old = recombinants[i]
            recombinants[i] = recombinants[i][:r] + str(m) + recombinants[i][r + 1:]
            #print(old, "->", recombinants[i])
        else:
            #print(recombinants[i])
            pass
    return(recombinants)

def solve_n_queens_genetic_algorithm(num_states, board_size):
    if board_size <= 10:
        print_frequency = 1000
    else:
        print_frequency = 100
    #DRIVER CODE
    states = generate_initial_states(num_states, board_size)
    new_states = random_state_selection(states) # randomly choose states from states
    recombinants = crossover(new_states) # do recombination
    recombinants = random_mutation(recombinants) # do random mutations
    next_generation = []
    winner = False
    solution = None
    generations = 0
    while not winner:
#         if generations % print_frequency == 0:
#             print("GENERATION:", generations)
#             print(recombinants)
        generations += 1
        for i in range(len(recombinants)):
            next_generation.append(string_to_board(recombinants[i]))
        next_generation = generate_successor_states(next_generation, board_size)
        next_generation
        new_states = random_state_selection(next_generation)
        for i in range(len(new_states)):
            if new_states[i][1] == int (max_attacking_pairs):
                solution = new_states[i][0]
                winner = True
                break
        recombinants = crossover(new_states)
        #recombinants = random_mutation(recombinants)
        recombinants = random_mutation_characterwise(recombinants)        
        next_generation = []
    print("LEVEL", str(board_size), "\nWINNER WINNER CHICKEN DINNER")
    print(np.matrix(solution))
    print("GENERATIONS:", generations, "\n")
    return solution

rec = ['Hello World!', 'Hello World!', 'Hello World!', 'Hello World!']
def random_mutation_characterwise(recombinants):
    count = 0
    total = 0
    mutants = []
    length = len(recombinants[0])
    #print(length)
    for r in recombinants:
        mutant = ""
        for i in range(len(r)):
            total += 1
            if random.randint(0, 6) == 0:
                char = str(random.randint(0, length - 1))
                #print(char, end=" ")
                mutant += char
                count += 1
            else:
                #print(r[i], end=" ")
                mutant += r[i]
        mutants.append(mutant)
        #print()
    ratio = 100 * count / total
    #print(ratio)
    #print(mutants)
    return mutants

In [5]:
for i in range(4, 11):
    solve_n_queens_genetic_algorithm(8, i)

LEVEL 4 
WINNER WINNER CHICKEN DINNER
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]
GENERATIONS: 12 

LEVEL 5 
WINNER WINNER CHICKEN DINNER
[[0 0 0 1 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]]
GENERATIONS: 125 

LEVEL 6 
WINNER WINNER CHICKEN DINNER
[[0 0 0 1 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 1 0 0 0]]
GENERATIONS: 1664 

LEVEL 7 
WINNER WINNER CHICKEN DINNER
[[0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0]]
GENERATIONS: 225 

LEVEL 8 
WINNER WINNER CHICKEN DINNER
[[0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]]
GENERATIONS: 4162 

LEVEL 9 
WINNER WINNER CHICKEN DINNER
[[0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0]]
GENERATIONS